In [ ]:
!pip install future

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install dedupe==1.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.9 MB/s 
     |████████████████████████████████| 3.9 MB 65.3 MB/s 
     |████████████████████████████████| 71 kB 9.5 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
     |████████████████████████████████| 640 kB 73.7 MB/s 
     |████████████████████████████████| 152 kB 69.2 MB/s 
     |████████████████████████████████| 105 kB 66.5 MB/s 
     |████████████████████████████████| 194 kB 64.3 MB/s 
     |████████████████████████████████| 260 kB 56.7 MB/s 
     |████████████████████████████████| 261 kB 69.7 MB/s 
     |████████████████████████████████| 1.1 MB 69.1 MB/s 
     |████████████████████████████████| 284 kB 81.6 MB/s 


In [ ]:
import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/data_for_ER/new_data.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
len(data)

968569

In [ ]:
data.sample(10)

,index,NAMELAST,NAMEFIRST,NAMEMID,UIN,BDGNBR,ACCESS_TYPE,TOA,POA,TOD,...,TERMINAL_SUFFIX,visitee_namelast,visitee_namefirst,MEETING_LOC,MEETING_ROOM,CALLER_NAME_LAST,CALLER_NAME_FIRST,CALLER_ROOM,Description,RELEASE_DATE
86338,86391,VRECENAK,MELANIE,I,U03087,NaN,VA,NaN,NaN,NaN,...,DC,OFFICE,VISITORS,WH,RESIDENCE,OFFICE,VISITORS,NaN,GROUP TOURS,8/27/2010
387069,387214,HANDLEY,DEBRA,NaN,U71048,NaN,VA,NaN,NaN,NaN,...,RC,OFFICE,VISITORS,WH,RESIDENCE,OFFICE,VISITORS,NaN,TOURS,4/30/2010
327514,327646,CLARK,RAY,NaN,U73296,78690.0,VA,2/19/2010 9:42,D1S01,2/19/2010 10:41,...,KS,MCPHILLIPS,RICHARD,OEOB,285,MCPHILLIPS,RICHARD,NaN,NaN,5/28/2010
715600,715921,BELL,DEVYN,G,U12258,NaN,VA,NaN,NaN,NaN,...,GB,OFFICE,VISITORS,WH,RESIDENCE,OFFICE,VISITORS,NaN,GROUP TOURS /,NaN
788213,788534,NASH,ELIAS,B,U21787,NaN,VA,NaN,NaN,NaN,...,RC,OFFICE,VISITORS,WH,RESIDENCE,OFFICE,VISITORS,NaN,GROUP TOURS /,10/29/2010
946941,947271,HUINER,HARVY,NaN,U41382,NaN,VA,NaN,NaN,NaN,...,B3,OFFICE,VISITORS,WH,RESIDENCE,OFFICE,VISITORS,NaN,TOURS,12/31/2010
898797,899126,STEWART,ZAHRA,NaN,U30392,NaN,VA,NaN,NaN,NaN,...,B3,OFFICE,VISITORS,WH,EW,OFFICE,VISITORS,NaN,TENNIS SERIES,11/26/2010
47305,47313,RUDERFER,STUART,NaN,U18376,74927.0,VA,6/22/2010 7:56,D1S01,6/22/2010 8:50,...,SL,RUSSELL,CATHERINE,OEOB,200,CAMPBELL,MARGARET,NaN,NaN,9/24/2010
855633,855960,GRAHAM,KENDRA,NaN,U34823,NaN,VA,NaN,NaN,NaN,...,GB,OFFICE,VISITORS,WH,RESIDENCE,OFFICE,VISITORS,NaN,GROUP TOUR,11/26/2010
635042,635316,MORGENSAI,MADISON,NaN,U45767,NaN,VA,NaN,NaN,NaN,...,P6,OFFICE,VISITORS,WH,RESIDENCE,OFFICE,VISITORS,NaN,ADD ON,1/29/2010


In [ ]:
import tempfile
import argparse
import csv
import os
import dedupe

In [ ]:
FIELDS =  [{'field': 'NAMEFIRST',
               'type': 'String','has missing': True},
              {'field': 'NAMELAST',
               'type': 'String','has missing': True},
              {'field': 'UIN',
               'type': 'String','has missing': True},
              {'field': 'MEETING_LOC',
               'type': 'String','has missing': True}
              ]

In [ ]:
%%time

# Create a new deduper object and pass our data model to it.
deduper = dedupe.Dedupe(FIELDS)

CPU times: user 479 µs, sys: 79 µs, total: 558 µs
Wall time: 565 µs


In [ ]:
%%time

import re

def preProcess(column):
  """
  Do a little bit of data cleaning with the help of Unidecode and Regex.
  Things like casing, extra spaces, quotes and new lines can be ignored.
  """
  column = re.sub('  +', ' ', column)
  column = re.sub('\n', ' ', column)
  column = column.strip().strip('"').strip("'").lower().strip()
  # If data is missing, indicate that by setting the value to `None`
  if not column:
    column = None
  return column

def readData(filename):
  """
  Read in our data from a CSV file and create a dictionary of records,
  where the key is a unique record ID and each value is dict
  """

  data_d = {}
  with open(filename) as f:
    reader = csv.DictReader(f)
    for row in reader:
      clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
      row_id = int(row['index'])
      data_d[row_id] = dict(clean_row)

  return data_d

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.9 µs


In [ ]:
%%time
data_d = readData("/content/drive/MyDrive/data_for_ER/new_data.csv")

CPU times: user 51.7 s, sys: 1.33 s, total: 53 s
Wall time: 53.1 s


In [ ]:
%%time
deduper.prepare_training(data_d)

CPU times: user 8min 45s, sys: 3.15 s, total: 8min 48s
Wall time: 8min 48s


In [ ]:
dedupe.consoleLabel(deduper)

NAMEFIRST : matthew
NAMELAST : leger
UIN : u61731
MEETING_LOC : wh

NAMEFIRST : matthew
NAMELAST : leger
UIN : u17731
MEETING_LOC : wh

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


1


(y)es / (n)o / (u)nsure / (f)inished


y


NAMEFIRST : judith
NAMELAST : matheny
UIN : u34185
MEETING_LOC : wh

NAMEFIRST : None
NAMELAST : katherine
UIN : None
MEETING_LOC : vpr

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

NAMEFIRST : judith
NAMELAST : payne
UIN : u07630
MEETING_LOC : wh

NAMEFIRST : judith
NAMELAST : payne
UIN : u98806
MEETING_LOC : wh

9/10 positive, 11/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


NAMEFIRST : tanisha
NAMELAST : dozier
UIN : u90977
MEETING_LOC : wh

NAMEFIRST : ladis
NAMELAST : dozier
UIN : u56433
MEETING_LOC : wh

10/10 positive, 11/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [ ]:
%%time

deduper.train()

CPU times: user 2.54 s, sys: 563 ms, total: 3.1 s
Wall time: 2.53 s
